In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
products = pd.read_csv('producto_tabla.csv')

In [1]:
train = pd.read_csv('train.csv', usecols=['Producto_ID', 'Venta_hoy', 'Venta_uni_hoy'])
train['price'] = train['Venta_hoy']/train['Venta_uni_hoy']
tp = train[['Producto_ID', 'price']]

In [2]:
price = tp.groupby(['Producto_ID'])['price'].mean().reset_index()

In [3]:
products = pd.merge(products, price, how='left', on=['Producto_ID'])

In [2]:
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'g':1})
products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000) 

product_bag_words = vectorizer.fit_transform(products.short_name).toarray()
product_bag_words.shape

(2592L, 808L)

In [4]:
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering

cltr = KMeans(n_clusters=100)
inds = cltr.fit_predict(product_bag_words)

In [5]:
products['short_name'].values[np.where(inds==0)]

array(['Tartinas Zarzamora ', 'Tartinas Zarzamora ', 'Tartinas Zarzamora ',
       'Tartina Zarzamora ', 'Tartinas Zarzamora ', 'Tartinas Zarzamora ',
       'Tartinas Zarzamora ', 'Tartinas Zarzamora ',
       'Tartinas Zarzamora MG ', 'Tartinas Zarzamora '], dtype=object)

In [7]:
products['cluster_nombre'] = inds
products['drink'] = 0

In [8]:
price_na = {}
for row in products.iterrows():
    if np.isnan(row[1]['weight']):
        products.set_value(row[0], 'drink', 1)
    if np.isnan(row[1]['price']):
        cluster = products[products['cluster_nombre']==row[1]['cluster_nombre']]
        subcluster = cluster[cluster['brand']==row[1]['brand']]
        subsubcluster = cluster[cluster['weight']==row[1]['weight']]
        subsubmed =  subsubcluster['price'].median()
        submed =  subcluster['price'].median()
        med =  cluster['price'].median()
        if np.isnan(subsubmed):
            if np.isnan(submed):
                if np.isnan(med):
                    products.set_value(row[0], 'price', 0)
                else:
                    products.set_value(row[0], 'price', med)
            else:
                products.set_value(row[0], 'price', submed)
        else:
            products.set_value(row[0], 'price', subsubmed)

In [9]:
products['pieces'].fillna(1, inplace=True)
products['w_per_piece'] = products['weight']/products['pieces']
products.fillna(0, inplace=True)

In [10]:
products.to_csv('products.csv')

In [ ]:
productss = pd.read_csv('products.csv')

In [11]:
products

,Producto_ID,NombreProducto,price,short_name,brand,weight,pieces,cluster_nombre,drink,w_per_piece
0,9,Capuccino Moka 750g NES 9,82.500000,Capuccino Moka,NES,750.0,1.0,93,0,750.000000
1,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,17.992632,Bimbollos Ext sAjonjoli,BIM,480.0,6.0,62,0,80.000000
2,53,Burritos Sincro 170g CU LON 53,14.300088,Burritos Sincro,LON,170.0,1.0,93,0,170.000000
3,72,Div Tira Mini Doradita 4p 45g TR 72,3.697305,Div Tira Mini Doradita,TR,45.0,4.0,44,0,11.250000
4,73,Pan Multigrano Linaza 540g BIM 73,21.360519,Pan Multigrano Linaza,BIM,540.0,1.0,75,0,540.000000
5,98,Tostado Integral 180g WON 98,15.182422,Tostado Integral,WON,180.0,1.0,56,0,180.000000
6,99,Pan Blanco 567g WON 99,16.591863,Pan Blanco,WON,567.0,1.0,26,0,567.000000
7,100,Super Pan Bco Ajonjoli 680g SP WON 100,20.388101,Super Pan Bco Ajonjoli,WON,680.0,1.0,2,0,680.000000
8,106,Wonder 100pct mediano 475g WON 106,19.142935,Wonder,WON,475.0,1.0,28,0,475.000000
9,107,Wonder 100pct gde 680g SP WON 107,21.320000,Wonder,WON,680.0,1.0,28,0,680.000000
